In [1]:
import datetime
import itertools
import os
import re
import requests
import sys
import time
from copy import deepcopy
from collections import namedtuple, defaultdict

from bs4 import BeautifulSoup as BS
from tenacity import retry, stop_after_attempt
import simplejson as json
from selenium import webdriver

from utils import ArticleMeta, NewsCrawler

In [2]:
class EttodayNewsCrawler(NewsCrawler):
    
    def date_to_newslist_url(self, date):
        
        return "https://www.ettoday.net/news/news-list-%s-0.htm" % date
    
    def unfold_abbr_newslink(self, url):
        
        return "https://www.ettoday.net" + url
    
    def newslink_generator(self):
        
        ettoday_date_form = re.compile("[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}")
        
        for date in self.date_generator():
            newslist_url = self.date_to_newslist_url(date)
            newslist = self.get_bsObj_scroll_down(newslist_url)
            news_datetimes = newslist.find_all('span', class_='date', text=ettoday_date_form)

            for news_datetime in news_datetimes:
                newslinks = news_datetime.parent.a['href']
                yield newslinks
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_category(self, newspage):
        return newspage.findAll('a', itemprop='item')[1].span.text
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_title(self, newspage):
        return newspage.find('h1', class_='title', itemprop='headline').text
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_article(self, newspage):
        paragraphs = []
        for paragraph in newspage.find('article').find_all('p'):
            if paragraph.findChild():
                continue
            else:
                paragraphs.append(paragraph.text)
        return '\n'.join(paragraphs)
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_date_time(self, newspage):
        # workaround
        datetime_string = newspage.find('time')['datetime'][:-6]
        dt = datetime.datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S")
        return str(dt.date()), str(dt.time())
    
    def is_valid_newspage(self, bsObj):
        
        if bsObj is None:
            return False
        
        try:
            if bsObj.find('em').text == '404錯誤':
                return False
            else:
                return True
        except:
            return True
    
    def saved_filename(self, url):
        return url.split('/')[-1].split('.')[0] + '.json'
    
    def crawl_and_save(self):
        
        for newslink in self.newslink_generator():
            article = self.get_page_attribute_from_link(self.unfold_abbr_newslink(newslink))
            
            if article is None:
                continue

            self.save_article_meta(article)

In [3]:
ettoday_crawler = EttodayNewsCrawler(
    output_dir='../news/ettoday',
    total_days=1,
    start_date=datetime.date(year=2019, month=7, day=31)
)

/home/mercedes/VirtualEnvs/SDML/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [4]:
ettoday_crawler.crawl_and_save()

Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502585.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1503060.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503037.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503057.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1503058.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503056.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1503055.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503053.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503022.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1503001.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503051.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1503047.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1503026.htm
Invalid

Successful Scraping URL: https://www.ettoday.net/news/20190731/1502912.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502957.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502956.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502770.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502834.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502946.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502515.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502950.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502948.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502944.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502890.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502433.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502940.htm
Inva

Successful Scraping URL: https://www.ettoday.net/news/20190731/1502631.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502243.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502808.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502847.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502836.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502811.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502828.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502325.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502855.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502745.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502478.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502850.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502678.htm
Invalid

Successful Scraping URL: https://www.ettoday.net/news/20190731/1502708.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502734.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502737.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502728.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502730.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502732.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502669.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502727.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502698.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502684.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502739.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502723.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502706.htm
Successful Scrap

Successful Scraping URL: https://www.ettoday.net/news/20190731/1502254.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502067.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502614.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502555.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502242.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502233.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502610.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502557.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502621.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1501524.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502589.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502562.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502258.htm
Invalid or

Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502443.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502492.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502470.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502404.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502194.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502489.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502477.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502428.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502469.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502482.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502425.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502438.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502393.htm
S

Successful Scraping URL: https://www.ettoday.net/news/20190731/1502382.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502334.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502371.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502386.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502385.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502332.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502326.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502380.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502227.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502247.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502231.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502368.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502369.htm
Invalid or Re

Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1463088.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502174.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502161.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1497486.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502175.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502169.htm
Invalid or Redirected Page: https://www.ettoday.net/news/20190731/1502173.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502171.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502113.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502112.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502170.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502160.htm
Successful Scraping URL: https://www.ettoday.net/news/20190731/1502168.htm
Successful